In [1]:
# Import libraries
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
# from sklearn.feature_extraction.text import CountVectorizer, BERTTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from collections import defaultdict
import re
import json
from sklearn.metrics import confusion_matrix
import pickle


# Initialization
lemmatizer = WordNetLemmatizer()
# count_vectorizer = CountVectorizer(stop_words="english", ngram_range=(1, 1))
# BERT_transformer = BERTTransformer()
nb_clf = MultinomialNB()

tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
tag_map['AS'] = wn.ADJ_SAT

# filepath = "finalized_8K_accounts.csv"
# filepath = "finalized_8K_accounts_emojis_replaced.csv"
# filepath = "FINALIZED_Training_Data_ALL_Available_Descriptions_EMOJIS_REPLACED.csv"
filepath = "FINALIZED_Training_Data_ALL_Available_Descriptions_EMOJIS_UNCHANGED.csv"

hand_label = "hand.label_simplified"

df = pd.read_csv(filepath)

# Removing all the "-int" (international, non-English, descriptions)
#dict.fromkeys(df[hand_label])
df = df[((df[hand_label] == 'media') | (df[hand_label] == 'tourbiz') |(df[hand_label] == 'acad') | (df[hand_label] == 'gov') | (
        df[hand_label] == 'other'))]

df = df[['username', 'description', hand_label]]  # keep only relevant columns

words_not_changed = ['media']

result = {}
n_gram_range = (1, 1)


def preprocessing(row):
    if str(row) == "nan":
        lemma = ""
    else:
        row = str(row).lower()
        row = word_tokenize(row)  # tokenize
        lemma = [lemmatizer.lemmatize(token, tag_map[tag[0]]) if token not in words_not_changed else token for
                 token, tag in pos_tag(row)]  # lemmatization, depending on part-of-speech
        lemma = ["" if re.search(r'\b[0-9]+\b\s*', lem) else lem for lem in lemma]  # removing
    return str(lemma)


df['description_lemmatized'] = df['description'].apply(preprocessing)

# Remove all the empty descriptions
df = df[df['description_lemmatized'] != ""]
#df[hand_label]
#print(df.shape)
#df[df['description_lemmatized'] != ""].shape

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

#Sentences are encoded by calling model.encode()
#print(type(df[['description']]))
embeddings = model.encode(df['description'].tolist())

In [3]:
import keras
import scikeras
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from keras.layers import Dense
#from keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasClassifier, KerasRegressor

2024-03-19 14:01:12.528232: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-19 14:01:12.868044: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/jvm/java-1.11.0-openjdk-amd64/lib/server:
2024-03-19 14:01:12.868063: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-19 14:01:13.970199: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.

In [71]:
# create a function that returns a model, taking as parameters things you
# want to verify using cross-valdiation and model selection
def create_model(neurons, optimizer='rmsprop', kernel_initializer='glorot_uniform'):
    model = Sequential()
    model.add(Dense(neurons,activation='relu',kernel_initializer=kernel_initializer, input_dim=384))
    model.add(Dense(neurons,activation='relu',kernel_initializer=kernel_initializer))
    #model.add(Dropout(dropout))
    #model.add(Dense(1,activation='sigmoid',kernel_initializer=kernel_initializer))
    model.add(Dense(units = 5, activation = 'softmax', kernel_initializer=kernel_initializer))

    model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

    return model

##Reproducibility
seed = np.random.seed(22)
rng = np.random.RandomState(1)

# wrap the model using the function you created
clf = KerasClassifier(build_fn=create_model, neurons = 10, random_state=rng, validation_split=0.2, verbose=False, callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=20)])

# just create the pipeline
pipeline = Pipeline([
    ('normalize', StandardScaler(with_mean=False)),
    ('clf',clf)
])

In [72]:
X = embeddings
y_labels = df[hand_label]
y_orig_labels = df[hand_label]

from sklearn.preprocessing import LabelBinarizer
label_as_binary = LabelBinarizer()
y_labels = label_as_binary.fit_transform(y_labels)

X_train, X_test, y_train, y_test = train_test_split(X, y_labels, test_size=0.2, random_state=42, stratify=y_labels)

In [79]:


## CV grid search
param_BERT_grid = [
    {
        'clf__neurons': [10, 50, 100, 200],
    }
]

# !!! Does STRATIFICATION BY DEFAULT !!!
grid_search_BERT = GridSearchCV(pipeline, param_BERT_grid, cv=10, scoring='accuracy', verbose=0)
grid_search_BERT.fit(X_train, y_train)

INFO:tensorflow:Assets written to: ram://49572e652e454f348538ff360bd79cea/assets


INFO:tensorflow:Assets written to: ram://49572e652e454f348538ff360bd79cea/assets
2024-03-19 16:49:16.078324: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://e4485f6213b843f98692f0f552d40aae: INVALID_ARGUMENT: ram://e4485f6213b843f98692f0f552d40aae is a directory.


INFO:tensorflow:Assets written to: ram://21f2e208d58a4222adc0e5303ff3cc6e/assets


INFO:tensorflow:Assets written to: ram://21f2e208d58a4222adc0e5303ff3cc6e/assets
2024-03-19 16:49:16.721101: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://11ec1068b5c342dcafd673555da15114: INVALID_ARGUMENT: ram://11ec1068b5c342dcafd673555da15114 is a directory.
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


INFO:tensorflow:Assets written to: ram://71c866cf337649518209c2dbf2d27c73/assets


...

INFO:tensorflow:Assets written to: ram://ff2793e733824e4d9279a608e4727a85/assets


INFO:tensorflow:Assets written to: ram://ff2793e733824e4d9279a608e4727a85/assets
2024-03-19 16:50:30.909596: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://155a241b44ed46e9a893adf87288430e: INVALID_ARGUMENT: ram://155a241b44ed46e9a893adf87288430e is a directory.
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


INFO:tensorflow:Assets written to: ram://51dfc4f21e3e475f984d4a962c4219b6/assets


INFO:tensorflow:Assets written to: ram://51dfc4f21e3e475f984d4a962c4219b6/assets
2024-03-19 16:50:32.689585: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://9c0a5861b1cf4a899d4369d44e4c4fa4: INVALID_ARGUMENT: ram://9c0a5861b1cf4a899d4369d44e4c4fa4 is a directory.
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('normalize',
                                        StandardScaler(with_mean=False)),
                                       ('clf',
                                        KerasClassifier(build_fn=<function create_model at 0x7fa31f7cf4c0>, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa31d3fcac0>], neurons=200, random_state=RandomState(MT19937) at 0x7FA3444D2340, validation_split=0.2, verbose=False))]),
             param_grid=[{'clf__neurons': [10, 50, 100, 200]}],
             scoring='accuracy')

In [80]:
grid_search_BERT.best_params_

{'clf__neurons': 100}

In [81]:
pipeline.set_params(**grid_search_BERT.best_params_)
our_model = pipeline.fit(X_train, y_train)

/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


In [82]:
y_pred_cv = cross_val_predict(our_model, X_train, y_train, cv=10)

INFO:tensorflow:Assets written to: ram://9285c9c4da734dc3b247e7d969ed0a23/assets


INFO:tensorflow:Assets written to: ram://9285c9c4da734dc3b247e7d969ed0a23/assets
2024-03-19 16:51:40.572695: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://136035d612a44702989e4f1f09c3b711: INVALID_ARGUMENT: ram://136035d612a44702989e4f1f09c3b711 is a directory.
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


INFO:tensorflow:Assets written to: ram://34e5f5a032814b0ebc52c992386d8c63/assets


INFO:tensorflow:Assets written to: ram://34e5f5a032814b0ebc52c992386d8c63/assets


2024-03-19 16:51:43.361705: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://1bdf278f94014d18afa85a8d9bd67dd8: INVALID_ARGUMENT: ram://1bdf278f94014d18afa85a8d9bd67dd8 is a directory.
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

INFO:tensorflow:Assets written to: ram://6f620f89b6f144a78f21e7aeb5ca1efc/assets
2024-03-19 16:51:54.955751: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://08f092b2f51747c2b79a383cb6d8551c: INVALID_ARGUMENT: ram://08f092b2f51747c2b79a383cb6d8551c is a directory.
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


INFO:tensorflow:Assets written to: ram://f167e5e923af4ab5bb085209a54a25fb/assets


INFO:tensorflow:Assets written to: ram://f167e5e923af4ab5bb085209a54a25fb/assets
2024-03-19 16:51:56.825772: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://3846900937ef472085fd550d85955310: INVALID_ARGUMENT: ram://3846900937ef472085fd550d85955310 is a directory.
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


INFO:tensorflow:Assets written to: ram://831db5938ade4a38bd955a6f26433119/assets


INFO:tensorflow:Assets written to: ram://831db5938ade4a38bd955a6f26433119/assets
2024-03-19 16:51:58.480427: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://5838b5f69a0741d7aa8c8dc143793c26: INVALID_ARGUMENT: ram://5838b5f69a0741d7aa8c8dc143793c26 is a directory.
/home/andrey/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


In [83]:
y_pred_cv_orig = label_as_binary.inverse_transform(y_pred_cv)
y_train_orig = label_as_binary.inverse_transform(y_train)

In [84]:
print(metrics.classification_report(y_train_orig, y_pred_cv_orig))

              precision    recall  f1-score   support

        acad       0.59      0.49      0.54       430
         gov       0.76      0.13      0.22       103
       media       0.78      0.73      0.76      1299
       other       0.91      0.95      0.93      7551
     tourbiz       0.70      0.29      0.41       155

    accuracy                           0.88      9538
   macro avg       0.75      0.52      0.57      9538
weighted avg       0.87      0.88      0.87      9538



In [85]:
y_test_pred = our_model.predict(X_test)
y_test_pred_orig = label_as_binary.inverse_transform(y_test_pred)

y_test_orig = label_as_binary.inverse_transform(y_test)

print(metrics.classification_report(y_test_orig, y_test_pred_orig))

              precision    recall  f1-score   support

        acad       0.73      0.37      0.49       108
         gov       0.00      0.00      0.00        25
       media       0.86      0.58      0.69       325
       other       0.88      0.98      0.92      1888
     tourbiz       0.78      0.18      0.29        39

    accuracy                           0.87      2385
   macro avg       0.65      0.42      0.48      2385
weighted avg       0.86      0.87      0.85      2385



/home/andrey/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andrey/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andrey/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
